In [1]:
# Copyright (c) 2021, InterDigital R&D France. All rights reserved.
#
# This source code is made available under the license found in the
# LICENSE.txt in the root directory of this source tree.
import matplotlib.pyplot as plt
%matplotlib inline

import argparse
import glob
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import yaml

from PIL import Image
from torchvision import transforms, utils, models

import sys

if os.getcwd().split('/')[-1] == 'notebooks':
    sys.path.append('..')
    os.chdir('..')
    
from datasets import *
from trainer import *
from utils.functions import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)
Image.MAX_IMAGE_PIXELS = None
device = torch.device('cuda')

parser = argparse.ArgumentParser()
parser.add_argument('--config', type=str, default='003', help='Path to the config file.')
parser.add_argument('--attr', type=str, default='Eyeglasses', help='attribute for manipulation.')
parser.add_argument('--latent_path', type=str, default='./data/celebahq_dlatents_psp.npy', help='dataset path')
parser.add_argument('--label_file', type=str, default='./data/celebahq_anno.npy', help='label file path')
parser.add_argument('--stylegan_model_path', type=str, default='./pixel2style2pixel/pretrained_models/psp_ffhq_encode.pt', help='stylegan model path')
parser.add_argument('--classifier_model_path', type=str, default='./models/latent_classifier_epoch_20.pth', help='pretrained attribute classifier')
parser.add_argument('--log_path', type=str, default='./logs/', help='log file path')
opts = parser.parse_args([])

# Celeba attribute list
attr_dict = {'5_o_Clock_Shadow': 0, 'Arched_Eyebrows': 1, 'Attractive': 2, 'Bags_Under_Eyes': 3, \
            'Bald': 4, 'Bangs': 5, 'Big_Lips': 6, 'Big_Nose': 7, 'Black_Hair': 8, 'Blond_Hair': 9, \
            'Blurry': 10, 'Brown_Hair': 11, 'Bushy_Eyebrows': 12, 'Chubby': 13, 'Double_Chin': 14, \
            'Eyeglasses': 15, 'Goatee': 16, 'Gray_Hair': 17, 'Heavy_Makeup': 18, 'High_Cheekbones': 19, \
            'Male': 20, 'Mouth_Slightly_Open': 21, 'Mustache': 22, 'Narrow_Eyes': 23, 'No_Beard': 24, \
            'Oval_Face': 25, 'Pale_Skin': 26, 'Pointy_Nose': 27, 'Receding_Hairline': 28, 'Rosy_Cheeks': 29, \
            'Sideburns': 30, 'Smiling': 31, 'Straight_Hair': 32, 'Wavy_Hair': 33, 'Wearing_Earrings': 34, \
            'Wearing_Hat': 35, 'Wearing_Lipstick': 36, 'Wearing_Necklace': 37, 'Wearing_Necktie': 38, 'Young': 39}

/home/adri/miniconda3/envs/lattrans/lib/python3.6/site-packages/torch/utils/cpp_extension.py:286: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) is not compatible with the compiler Pytorch was
built with for this platform, which is g++ on linux. Please
use g++ to to compile your extension. Alternatively, you may
compile PyTorch from source using c++, and then you can also use
c++ to compile your extension.

See https://github.com/pytorch/pytorch/blob/master/CONTRIBUTING.md for help
with compiling PyTorch from source.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  platform=sys.platform))


OSError: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.

In [ ]:
# Initialize trainer model.
log_dir = os.path.join(opts.log_path, opts.config) + '/'
config = yaml.load(open('./configs/' + opts.config + '.yaml', 'r'))

trainer = Trainer(config, None, None, opts.label_file)
trainer.initialize(opts.stylegan_model_path, opts.classifier_model_path)   
trainer.to(device)

testdata_dir = './data/supple/'
latent_list = glob.glob1(testdata_dir,'*.npy')
latent_list.sort()

attrs_list = [{'Arched_Eyebrows':1, 'Chubby':-0.5, 'Pointy_Nose':-1, 'Smiling':-1, 'Pale_Skin':1}, \
                {'Black_Hair':1, 'Smiling':1, 'Bags_Under_Eyes':1, 'Young':1, 'Eyeglasses':1.2}, \
                {'Chubby':-0.5, 'Bangs':1, 'Smiling':1, 'Young':-1, 'Heavy_Makeup':1}, \
                {'Chubby':-0.5, 'Smiling':-1, 'Narrow_Eyes':-0.5, 'Heavy_Makeup':1, 'Eyeglasses':1}, \
                {'Eyeglasses':-1.2, 'Smiling':1, 'Goatee':1, 'Arched_Eyebrows':1, 'Young':1}, \
                {'No_Beard':-1, 'Bushy_Eyebrows':1, 'Mouth_Slightly_Open':1, 'Receding_Hairline':1, 'Eyeglasses':1.2}, \
                {'Smiling':-1, 'Chubby':-0.5, 'No_Beard':-1, 'Eyeglasses':1.2, 'Receding_Hairline':1}, \
                {'Smiling':1, 'Goatee':1, 'Eyeglasses':-1, 'Arched_Eyebrows':-1, 'Young':1}  ]

### Single attribute manipulation

In [ ]:
%matplotlib inline
with torch.no_grad():
    
    for k, latent in enumerate(latent_list):

        w_0 = np.load(testdata_dir + latent)
        w_0 = torch.tensor(w_0).to(device)
        
        idx =  [int(s) for s in latent.replace('_','.').split('.') if s.isdigit()][0]
        x_0 = img_to_tensor(Image.open(testdata_dir + '%05d.jpg'%idx))
        x_0 = x_0.unsqueeze(0).to(device)
        img_l = [x_0] # original image
        
        x_1, _ = trainer.StyleGAN([w_0], input_is_latent=True, randomize_noise=False)
        x_0 = torch.ones((x_1.size(0), x_1.size(1), x_1.size(2),x_1.size(3)+40)).type_as(x_1)
        x_0[:,:,:,20:1044] = x_1[:,:,:,:]
        img_l.append(x_0) # projected image
        
        attrs = attrs_list[k]
        for attr in list(attrs.keys()):
            
            trainer.attr_num = attr_dict[attr]
            trainer.load_model(log_dir)
            
            alpha = torch.tensor(1.0) * attrs[attr]
            w_1 = trainer.T_net(w_0.view(w_0.size(0), -1), alpha.unsqueeze(0).to(device))
            w_1 = w_1.view(w_0.size())
            w_1 = torch.cat((w_1[:,:11,:], w_0[:,11:,:]), 1)
            x_1, _ = trainer.StyleGAN([w_1], input_is_latent=True, randomize_noise=False)
            img_l.append(x_1.data)

        img = img_l[0] if len(img_l)==1 else torch.cat(img_l, 3)
        img = np.clip(clip_img(img)[0].cpu().numpy()*255.,0,255).astype(np.uint8)
        img = Image.fromarray(img.transpose(1,2,0))
        plt.figure(figsize=(30,5))
        plt.imshow(img)
        plt.axis('off')
        plt.show()

### Sequential attribute manipulation

In [ ]:
with torch.no_grad():
    
    for k, latent in enumerate(latent_list):

        w_0 = np.load(testdata_dir + latent)
        w_0 = torch.tensor(w_0).to(device)
        
        idx =  [int(s) for s in latent.replace('_','.').split('.') if s.isdigit()][0]
        x_0 = img_to_tensor(Image.open(testdata_dir + '%05d.jpg'%idx))
        x_0 = x_0.unsqueeze(0).to(device)
        img_l = [x_0] # original image
        
        x_1, _ = trainer.StyleGAN([w_0], input_is_latent=True, randomize_noise=False)
        x_0 = torch.ones((x_1.size(0), x_1.size(1), x_1.size(2),x_1.size(3)+40)).type_as(x_1)
        x_0[:,:,:,20:1044] = x_1[:,:,:,:]
        img_l.append(x_0) # projected image
        
        attrs = attrs_list[k]
        w_1 = w_0
        for attr in list(attrs.keys()):
            
            trainer.attr_num = attr_dict[attr]
            trainer.load_model(log_dir)
            
            alpha = torch.tensor(1.0) * attrs[attr]
            w_1 = trainer.T_net(w_1.view(w_0.size(0), -1), alpha.unsqueeze(0).to(device))
            w_1 = w_1.view(w_0.size())
            w_1 = torch.cat((w_1[:,:11,:], w_0[:,11:,:]), 1)
            x_1, _ = trainer.StyleGAN([w_1], input_is_latent=True, randomize_noise=False)
            img_l.append(x_1.data)

        img = img_l[0] if len(img_l)==1 else torch.cat(img_l, 3)
        img = np.clip(clip_img(img)[0].cpu().numpy()*255.,0,255).astype(np.uint8)
        img = Image.fromarray(img.transpose(1,2,0))
        plt.figure(figsize=(30,5))
        plt.imshow(img)
        plt.axis('off')
        plt.show()